In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import math
from torch.utils.data import Dataset, DataLoader, TensorDataset
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import defaultdict
import random
import gc
from collections import Counter, OrderedDict
import torchtext
from torchtext.vocab import vocab as Vocab
from torch.optim import Adam
import time
import warnings
import plotly.express as px

warnings.filterwarnings("ignore")
torch.set_warn_always(False)
torchtext.disable_torchtext_deprecation_warning()

# BERT model setup
device = 'mps' if torch.cuda.is_available() else 'cpu'

data_dirs = {
    'dataset1': '../data/Loghub-BGL/processed/',
    'dataset2': '../data/Loghub-HPC/processed/'
}

# Configurations
window_size = 128
adaptive_window = True
seq_len = 512
max_len = 512
min_len = 10
mask_ratio = 0.5
train_ratio = 1
valid_ratio = 0.1
test_ratio = 1

corpus_lines = None
on_memory = True
batch_size = 32
num_workers = 5
learning_rate = 0.001
adam_beta1 = 0.9
adam_beta2 = 0.999
adam_weight_decay = 0.00
with_cuda = True
cuda_devices = None
log_freq = None
epochs = 200
hidden = 256 # embedding size
layers = 4
attn_heads = 4
is_logkey = True
is_time = False
scale = None
scale_path = None
n_epochs_stop = 10

/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


The following code uses BGL dataset to train BERT model

In [3]:
from sample import generate_train_valid

# BGL training data
train_path = '../data/Loghub-BGL/processed/train'

logkey_train, logkey_valid, time_train, time_valid = generate_train_valid(train_path, window_size=window_size,
                                  adaptive_window=adaptive_window,
                                  valid_size=valid_ratio,
                                  sample_ratio=train_ratio,
                                  scale=scale,
                                  scale_path=scale_path,
                                  seq_len=seq_len,
                                  min_len=min_len
                                )

num session: 13718
before filtering short session
train size  12346
valid size  1372


100%|██████████| 13718/13718 [00:02<00:00, 4848.54it/s]

Num of train seqs 5309
Num of valid seqs 1372


In [4]:
from log import LogDataset
from vocab import TorchVocab

datapath = '../data/Loghub-BGL/processed/train'
torch_vocab = TorchVocab(datapath=datapath)
vocab = torch_vocab.get_vocab()
train_dataset = LogDataset(logkey_train, time_train, vocab, seq_len=seq_len, corpus_lines=corpus_lines, on_memory=on_memory, mask_ratio=mask_ratio)
valid_dataset = LogDataset(logkey_valid, time_valid, vocab, seq_len=seq_len, on_memory=on_memory, mask_ratio=mask_ratio)
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers,
                                  collate_fn=train_dataset.collate_fn, drop_last=True)
valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=num_workers,
                                collate_fn=train_dataset.collate_fn, drop_last=True)

In [7]:
from model import BERT
from log import BERTLog

# BERT model
bert = BERT(len(vocab), max_len=max_len, hidden=hidden, n_layers=layers, attn_heads=attn_heads, is_logkey=is_logkey, is_time=is_time)

# Training
optimizer = Adam(bert.parameters(), lr=learning_rate, betas=(adam_beta1, adam_beta2), weight_decay=adam_weight_decay)
vocab_size = len(vocab)
# BERT language model
model = BERTLog(bert, vocab_size)
model.to(device)
# Using Negative Log Likelihood Loss function for predicting the masked_token
criterion = nn.NLLLoss(ignore_index=0)

In [6]:
def iteration(epoch, data_loader, start_train):
  total_dist = []
  total_loss = 0.0
  total_length = len(data_loader)
  data_iter = enumerate(data_loader)

  str_code = 'train' if start_train else 'valid'

  start = time.strftime("%H:%M:%S")
  start_time = time.time()

  for i, data in data_iter:
      data = {key: value.to(device) for key, value in data.items()}

      result = model.forward(data["bert_input"], data["time_input"])
      mask_lm_output, mask_time_output = result["logkey_output"], result["time_output"]

      # NLLLoss of predicting masked token word ignore_index = 0 to ignore unmasked tokens
      loss = torch.tensor(0) if not is_logkey else criterion(mask_lm_output.transpose(1, 2), data["bert_label"])

      total_loss += loss.item()

      # backward and optimization for training run only
      if (start_train):
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

  avg_loss = total_loss / total_length
  epoch_time = time.time() - start_time
  if start_train:
      train_loss.append({"epoch": epoch, "lr": learning_rate, "time": start, "loss": avg_loss})
      print(f'Epoch: {epoch + 1}, Train Loss: {avg_loss:.4f}, Time: {epoch_time:.2f} sec')
  else:
      valid_loss.append({"epoch": epoch, "lr": learning_rate, "time": start, "loss": avg_loss})
      print(f'Epoch: {epoch + 1}, Valid Loss: {avg_loss:.4f}, Time: {epoch_time:.2f} sec')

  return avg_loss, total_dist

def train(epoch):
  return iteration(epoch, train_data_loader, start_train=True)

def valid(epoch):
  return iteration(epoch, valid_data_loader, start_train=False)

In [6]:
print("Training Start")
for epoch in range(epochs):
    _, train_dist = train(epoch)
    avg_loss, valid_dist = valid(epoch)

torch.save(model.state_dict(), '../model-output/bert_model_dataset1.pth')

Training Start


/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: Us

KeyboardInterrupt: 

In [25]:
train_loss_df = pd.DataFrame(train_loss)
valid_loss_df = pd.DataFrame(valid_loss)

train_loss_df['type'] = 'Train'
valid_loss_df['type'] = 'Valid'
combined_loss_df = pd.concat([train_loss_df, valid_loss_df])
fig = px.line(combined_loss_df, x='epoch', y='loss', color='type', title='Epoch vs Loss (Train vs Valid)')
fig.show()

Results from training BERT model on BGL dataset

The following code uses System 20 dataset to train BERT model

In [7]:
from sample import generate_train_valid, generate_test
from log import LogDataset

train_path = '../data/Loghub-HPC/processed/train'
with open(train_path, 'r') as f:
    texts = f.readlines()

counter = Counter()
for line in texts:
    if isinstance(line, list):
        words = line
    else:
        words = line.replace("\n", "").replace("\t", "").split()
    for word in words:
        counter[word] += 1

specials=["<pad>", "<unk>", "<eos>", "<sos>", "<mask>"]
for tok in specials:
  del counter[tok]

sorted_by_freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)
vocab = Vocab(ordered_dict)
vocab.insert_token("<pad>", 0)
vocab.insert_token("<unk>", 1)
vocab.insert_token("<eos>", 2)
vocab.insert_token("<sos>", 3)
vocab.insert_token("<mask>", 4)
vocab.set_default_index(vocab["<unk>"])

logkey_train, logkey_valid, time_train, time_valid = generate_train_valid(train_path, window_size=window_size,
                                  adaptive_window=adaptive_window,
                                  valid_size=valid_ratio,
                                  sample_ratio=train_ratio,
                                  scale=scale,
                                  scale_path=scale_path,
                                  seq_len=seq_len,
                                  min_len=min_len
                                )

train_dataset = LogDataset(logkey_train, time_train, vocab, seq_len=seq_len, corpus_lines=corpus_lines, on_memory=on_memory, mask_ratio=mask_ratio)
valid_dataset = LogDataset(logkey_valid, time_valid, vocab, seq_len=seq_len, on_memory=on_memory, mask_ratio=mask_ratio)
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers,
                                  collate_fn=train_dataset.collate_fn, drop_last=True)
valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=num_workers,
                                collate_fn=train_dataset.collate_fn, drop_last=True)
del train_dataset
del valid_dataset
del logkey_valid
del logkey_train
del time_train
del time_valid
gc.collect()

num session: 20256
before filtering short session
train size  18230
valid size  2026


100%|██████████| 20256/20256 [00:00<00:00, 312312.10it/s]

Num of train seqs 13852
Num of valid seqs 2026


695

In [12]:
from model import BERT
from log import BERTLog 

# Training model on System 20 dataset
bert = BERT(len(vocab), max_len=max_len, hidden=hidden, n_layers=layers, attn_heads=attn_heads, is_logkey=is_logkey, is_time=is_time)
optimizer = Adam(bert.parameters(), lr=learning_rate, betas=(adam_beta1, adam_beta2), weight_decay=adam_weight_decay)
vocab_size = len(vocab)
criterion = nn.NLLLoss(ignore_index=0)

model = BERTLog(bert, vocab_size).to(device)

In [20]:
epochs = 50
print("Training Start")
for epoch in range(epochs):
  _, train_dist = train(epoch)
  avg_loss, valid_dist = valid(epoch)

torch.save(model.state_dict(), '../model-output/bert_model_dataset2.pth')

Training Start


/Users/allison/anaconda3/lib/python3.11/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/anaconda3/lib/python3.11/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/anaconda3/lib/python3.11/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TO

Epoch: 1, Train Loss: 0.4814, Time: 50.65 sec


/Users/allison/anaconda3/lib/python3.11/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/anaconda3/lib/python3.11/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/anaconda3/lib/python3.11/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TO

Epoch: 1, Valid Loss: 0.4742, Time: 26.45 sec


...

Epoch: 49, Valid Loss: 0.3904, Time: 26.65 sec


/Users/allison/anaconda3/lib/python3.11/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/anaconda3/lib/python3.11/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/anaconda3/lib/python3.11/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TO

Epoch: 50, Train Loss: 0.3881, Time: 51.19 sec


/Users/allison/anaconda3/lib/python3.11/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/anaconda3/lib/python3.11/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/anaconda3/lib/python3.11/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TO

Epoch: 50, Valid Loss: 0.3998, Time: 26.41 sec


In [21]:
train_loss_df = pd.DataFrame(train_loss)
valid_loss_df = pd.DataFrame(valid_loss)

train_loss_df['type'] = 'Train'
valid_loss_df['type'] = 'Valid'
combined_loss_df = pd.concat([train_loss_df, valid_loss_df])
fig = px.line(combined_loss_df, x='epoch', y='loss', color='type', title='Epoch vs Loss (Train vs Valid)')
fig.show()

In [11]:
import torch.nn.functional as F
from torch.utils.data import DataLoader
from collections import defaultdict

def detect_logkey_anomaly(masked_output, masked_label, num_candidates):
    num_undetected_tokens = 0
    output_maskes = []
    for i, token in enumerate(masked_label):
        output_maskes.append(torch.argsort(-masked_output[i])[:num_candidates].cpu().numpy())
        if token not in torch.argsort(-masked_output[i])[:num_candidates]:
            num_undetected_tokens += 1
    return num_undetected_tokens, [output_maskes, masked_label.cpu().numpy()]

def helper(model, data_loader, vocab, num_candidates, device):
    total_results = []
    output_results = []

    for data in data_loader:
        data = {key: value.to(device) for key, value in data.items()}

        result = model(data["bert_input"], data["time_input"])
        mask_lm_output = result["logkey_output"]

        # Loop through each session in batch
        for i in range(len(data["bert_label"])):
            seq_results = {"num_error": 0,
                           "undetected_tokens": 0,
                           "masked_tokens": 0,
                           "total_logkey": torch.sum(data["bert_input"][i] > 0).item()
                           }

            mask_index = data["bert_label"][i] > 0
            num_masked = torch.sum(mask_index).tolist()
            seq_results["masked_tokens"] = num_masked

            num_undetected, output_seq = detect_logkey_anomaly(mask_lm_output[i][mask_index], data["bert_label"][i][mask_index], num_candidates)
            seq_results["undetected_tokens"] = num_undetected
            output_results.append(output_seq)

            total_results.append(seq_results)

    return total_results, output_results

def compute_anomaly(results, seq_threshold=0.5):
    total_errors = 0
    for seq_res in results:
        if seq_res["undetected_tokens"] > seq_res["masked_tokens"] * seq_threshold:
            total_errors += 1
    return total_errors

def find_best_threshold(test_normal_results, test_abnormal_results, th_range, seq_range):
    best_result = [0] * 9
    for seq_th in seq_range:
        FP = compute_anomaly(test_normal_results, seq_th)
        TP = compute_anomaly(test_abnormal_results, seq_th)

        if TP == 0:
            continue

        TN = len(test_normal_results) - FP
        FN = len(test_abnormal_results) - TP
        P = 100 * TP / (TP + FP)
        R = 100 * TP / (TP + FN)
        F1 = 2 * P * R / (P + R)

        if F1 > best_result[-1]:
            best_result = [0, seq_th, FP, TP, TN, FN, P, R, F1]
    return best_result

def evaluate_model_on_test_data(model, data_loaders, vocab, device, num_candidates=15, seq_range=np.arange(0, 1, 0.1)):
    print("Testing normal data")
    test_normal_results, test_normal_errors = helper(model, data_loaders['normal'], vocab, num_candidates, device)

    print("Testing abnormal data")
    test_abnormal_results, test_abnormal_errors = helper(model, data_loaders['abnormal'], vocab, num_candidates, device)

    best_result = find_best_threshold(test_normal_results, test_abnormal_results, th_range=np.arange(10), seq_range=seq_range)

    best_th, best_seq_th, FP, TP, TN, FN, P, R, F1 = best_result

    accuracy = 100 * (TP + TN) / (TP + FP + TN + FN) if (TP + FP + TN + FN) > 0 else 0
    return accuracy, P, R, F1

In [10]:
from sample import generate_test 
from log import LogDataset

def evaluate(model, data_dir):
    test_norm_log_seqs, test_norm_tim_seqs = generate_test(data_dir + "test_normal", window_size=window_size, adaptive_window=adaptive_window, seq_len=seq_len, min_len=min_len)
    test_anom_log_seqs, test_anom_tim_seqs = generate_test(data_dir + "test_abnormal", window_size=window_size, adaptive_window=adaptive_window, seq_len=seq_len, min_len=min_len)
    test_norm_dataset = LogDataset(test_norm_log_seqs, test_norm_tim_seqs, vocab, seq_len, on_memory=on_memory, mask_ratio=mask_ratio)
    test_anom_dataset = LogDataset(test_anom_log_seqs, test_anom_tim_seqs, vocab, seq_len, on_memory=on_memory, mask_ratio=mask_ratio)

    test_normal_loader = DataLoader(test_norm_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=test_norm_dataset.collate_fn, drop_last=True)
    test_anomaly_loader = DataLoader(test_anom_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=test_anom_dataset.collate_fn, drop_last=True)
    data_loaders = {'normal': test_normal_loader, 'abnormal': test_anomaly_loader}

    accuracy, precision, recall, f1_score = evaluate_model_on_test_data(
        model, data_loaders, vocab, device='cpu'
    )
    return accuracy, precision, recall, f1_score

In [18]:
from model import BERT 
from log import BERTLog 
from vocab import TorchVocab

model1_dir = '../model-output/bert_model_dataset1.pth'
datapath = '../data/Loghub-BGL/processed/train'
torch_vocab = TorchVocab(datapath=datapath)
vocab = torch_vocab.get_vocab()
bert = BERT(len(vocab), max_len=max_len, hidden=hidden, n_layers=layers, attn_heads=attn_heads, is_logkey=is_logkey, is_time=is_time, additive=False)
vocab_size = len(vocab)
model1 = BERTLog(bert, vocab_size)
model1.load_state_dict(torch.load(model1_dir, map_location=torch.device('cpu')))

<All keys matched successfully>

In [19]:
accuracy, P, R, F1 = evaluate(model=model1, data_dir='../data/Loghub-BGL/processed/')

20579it [00:04, 5072.99it/s]
3018it [00:00, 4118.06it/s]

Testing normal data



/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: U

context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
cont

/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: Us

context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
cont

In [20]:
print("BERT model on BGL data results:")
print(f"Accuracy: {accuracy:.2f}, Precision: {P:.2f}, Recall: {R:.2f}, F1 Score: {F1:.2f}")

BERT model on BGL data results:
Accuracy: 95.61, Precision: 94.61, Recall: 83.57, F1 Score: 88.75


In [21]:
from model import BERT 
from log import BERTLog 
from vocab import TorchVocab

model2_dir = '../model-output/bert_model_dataset2.pth'
datapath = '../data/Loghub-HPC/processed/train'
torch_vocab = TorchVocab(datapath=datapath)
vocab = torch_vocab.get_vocab()
bert = BERT(len(vocab), max_len=max_len, hidden=hidden, n_layers=layers, attn_heads=attn_heads, is_logkey=is_logkey, is_time=is_time, additive=False)
vocab_size = len(vocab)
model2 = BERTLog(bert, vocab_size)
model2.load_state_dict(torch.load(model2_dir, map_location=torch.device('cpu')))

<All keys matched successfully>

In [22]:
accuracy, P, R, F1 = evaluate(model=model2, data_dir='../data/Loghub-HPC/processed/')

26732it [00:00, 266411.32it/s]
6087it [00:00, 278646.74it/s]

Testing normal data



/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: U

context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 1

/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: Us

context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
context torch.Size([32, 4, 1

In [23]:
print("BERT model on System 20 dataset results:")
print(f"Accuracy: {accuracy:.2f}, Precision: {P:.2f}, Recall: {R:.2f}, F1 Score: {F1:.2f}")

BERT model on System 20 dataset results:
Accuracy: 80.12, Precision: 68.97, Recall: 1.54, F1 Score: 3.02


In [7]:
num_candidates = 15
gaussian_mean = 0
gaussian_std = 1
model_dir = '../model-output/attn_model_dataset1.pth'
datapath = '../data/Loghub-HPC/processed/train'
torch_vocab = TorchVocab(datapath=datapath)
vocab = torch_vocab.get_vocab()

bert = BERT(len(vocab), max_len=max_len, hidden=hidden, n_layers=layers, attn_heads=attn_heads, is_logkey=is_logkey, is_time=is_time, additive=True)
vocab_size = len(vocab)
model = BERTLog(bert, vocab_size)
model.load_state_dict(torch.load(model_dir), strict=False)
warnings.filterwarnings("ignore")
accuracy, P, R, F1 = evaluate(model=model, data_dir='../data/Loghub-BGL/processed/')

Additive Attention model on BGL data results:


20579it [00:29, 704.86it/s] 
3018it [00:00, 4153.64it/s]

Testing normal data



/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: U

p_attn torch.Size([32, 4, 512, 64])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
p_attn torch.Size([32, 4, 512, 64])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
p_attn torch.Size([32, 4, 512, 64])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
p_attn torch.Size([32, 4, 512, 64])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
p_attn torch.Size([32, 4, 512, 64])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
p_attn torch.Size([32, 4, 512, 64])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
p_attn torch.Size([32, 4, 512, 64])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
p_attn torch.Size([32, 4, 512, 64])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
p_attn torch.Size([32, 4, 512, 64])
context torc

/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: Us

p_attn torch.Size([32, 4, 512, 64])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
p_attn torch.Size([32, 4, 512, 64])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
p_attn torch.Size([32, 4, 512, 64])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
p_attn torch.Size([32, 4, 512, 64])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
p_attn torch.Size([32, 4, 512, 64])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
p_attn torch.Size([32, 4, 512, 64])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
p_attn torch.Size([32, 4, 512, 64])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
p_attn torch.Size([32, 4, 512, 64])
context torch.Size([32, 4, 512, 64])
multi-head x shape torch.Size([32, 512, 256])
p_attn torch.Size([32, 4, 512, 64])
context torc

In [9]:
print("Additive Attention model on BGL data results:")
print(f'Accuracy: {accuracy:.4f}, Precision: {P:.4f}, Recall: {R:.4f}, F1-Score: {F1:.4f}')

Additive Attention model on BGL data results:
Accuracy: 88.2812, Precision: 72.3421, Recall: 70.2744, F1-Score: 71.2933


In [27]:
model_dir = '../model-output/attn_model_dataset2.pth'
datapath = '../data/Loghub-HPC/processed/train'
torch_vocab = TorchVocab(datapath=datapath)
vocab = torch_vocab.get_vocab()
bert = BERT(len(vocab), max_len=max_len, hidden=hidden, n_layers=layers, attn_heads=attn_heads, is_logkey=is_logkey, is_time=is_time, additive=True)
vocab_size = len(vocab)
model = BERTLog(bert, vocab_size)
model.load_state_dict(torch.load(model_dir, map_location=torch.device('cpu')))
accuracy, P, R, F1 = evaluate(model=model, data_dir='../data/Loghub-HPC/processed/')

26732it [00:00, 277190.41it/s]
6087it [00:00, 263872.59it/s]

Testing normal data



/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: U

p_attn torch.Size([32, 4, 11, 64])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
p_attn torch.Size([32, 4, 11, 64])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
p_attn torch.Size([32, 4, 11, 64])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
p_attn torch.Size([32, 4, 11, 64])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
p_attn torch.Size([32, 4, 11, 64])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
p_attn torch.Size([32, 4, 11, 64])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
p_attn torch.Size([32, 4, 11, 64])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
p_attn torch.Size([32, 4, 11, 64])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
p_attn torch.Size([32, 4, 11, 64])
context torch.Size([32, 4, 11, 64])
m

/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: Us

p_attn torch.Size([32, 4, 11, 64])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
p_attn torch.Size([32, 4, 11, 64])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
p_attn torch.Size([32, 4, 11, 64])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
p_attn torch.Size([32, 4, 11, 64])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
p_attn torch.Size([32, 4, 11, 64])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
p_attn torch.Size([32, 4, 11, 64])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
p_attn torch.Size([32, 4, 11, 64])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
p_attn torch.Size([32, 4, 11, 64])
context torch.Size([32, 4, 11, 64])
multi-head x shape torch.Size([32, 11, 256])
p_attn torch.Size([32, 4, 11, 64])
context torch.Size([32, 4, 11, 64])
m

In [28]:
print("Additive Attention model on System 20 data results:")
print(f'Accuracy: {accuracy:.4f}, Precision: {P:.4f}, Recall: {R:.4f}, F1-Score: {F1:.4f}')

Additive Attention model on System 20 data results:
Accuracy: 78.1443, Precision: 43.8113, Recall: 31.8866, F1-Score: 36.9097


In [4]:
# Heirarchical Attention Mechanism
from model import BERT 
from log import BERTLog 
from train import Trainer
from vocab import TorchVocab
from sample import generate_train_valid
from sample import generate_train_valid, generate_test
from log import LogDataset

train_path = '../data/Loghub-BGL/processed/train'
torch_vocab = TorchVocab(datapath=train_path)
vocab = torch_vocab.get_vocab()
# Train and validation sets
logkey_train, logkey_valid, time_train, time_valid = generate_train_valid(train_path, window_size=window_size,
                                  adaptive_window=adaptive_window,
                                  valid_size=valid_ratio,
                                  sample_ratio=train_ratio,
                                  scale=scale,
                                  scale_path=scale_path,
                                  seq_len=seq_len,
                                  min_len=min_len
                                )

train_dataset = LogDataset(logkey_train, time_train, vocab, seq_len=seq_len, corpus_lines=corpus_lines, on_memory=on_memory, mask_ratio=mask_ratio)
valid_dataset = LogDataset(logkey_valid, time_valid, vocab, seq_len=seq_len, on_memory=on_memory, mask_ratio=mask_ratio)
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers,
                                  collate_fn=train_dataset.collate_fn, drop_last=True)
valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=num_workers,
                                collate_fn=train_dataset.collate_fn, drop_last=True)

# BERT model
bert = BERT(len(vocab), max_len=max_len, hidden=hidden, n_layers=layers, attn_heads=attn_heads, is_logkey=is_logkey, is_time=is_time, attn='hierarchical')
optimizer = Adam(bert.parameters(), lr=learning_rate, betas=(adam_beta1, adam_beta2), weight_decay=adam_weight_decay)
vocab_size = len(vocab)
criterion = nn.NLLLoss(ignore_index=0)
model = BERTLog(bert, vocab_size).to(device)
train_loss = []
valid_loss = []
print("Training Start")
trainer = Trainer(model, device, is_logkey=is_logkey, criterion=criterion, lr=learning_rate, optim=optimizer, train_data_loader=train_data_loader, valid_data_loader=valid_data_loader)
epochs = 20
for epoch in range(epochs):
  _, train_dist = trainer.train(epoch)
  avg_loss, valid_dist = trainer.valid(epoch)

torch.save(model.state_dict(), '../model-output/h_attn_model_dataset1.pth')

num session: 13718
before filtering short session
train size  12346
valid size  1372


100%|██████████| 13718/13718 [00:28<00:00, 489.19it/s] 

Num of train seqs 5309
Num of valid seqs 1372
Training Start



/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: U

Epoch: 1, Train Loss: 3.0240, Time: 162.13 sec


...

Epoch: 19, Valid Loss: 1.7953, Time: 41.31 sec


/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: Us

Epoch: 20, Train Loss: 1.1791, Time: 166.13 sec


/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: Us

Epoch: 20, Valid Loss: 1.7394, Time: 41.30 sec


In [12]:
model_dir = '../model-output/h_attn_model_dataset1.pth'
datapath = '../data/Loghub-BGL/processed/train'
torch_vocab = TorchVocab(datapath=datapath)
vocab = torch_vocab.get_vocab()
bert = BERT(len(vocab), max_len=max_len, hidden=hidden, n_layers=layers, attn_heads=attn_heads, is_logkey=is_logkey, is_time=is_time, attn='hierarchical')
vocab_size = len(vocab)
model = BERTLog(bert, vocab_size)
model.load_state_dict(torch.load(model_dir, map_location=torch.device('cpu')))
accuracy, P, R, F1 = evaluate(model=model, data_dir='../data/Loghub-BGL/processed/')

20579it [00:54, 378.31it/s] 
3018it [00:00, 4133.49it/s]

Testing normal data



/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: U

Testing abnormal data


/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: Us

In [13]:
print("BERT model with Hierarchical Attention on BGL data results:")
print(f"Accuracy: {accuracy:.2f}, Precision: {P:.2f}, Recall: {R:.2f}, F1 Score: {F1:.2f}")

BERT model with Hierarchical Attention on BGL data results:
Accuracy: 93.17, Precision: 86.30, Recall: 79.69, F1 Score: 82.86


In [14]:
train_path = '../data/Loghub-HPC/processed/train'
torch_vocab = TorchVocab(datapath=train_path)
vocab = torch_vocab.get_vocab()
# Train and validation sets
logkey_train, logkey_valid, time_train, time_valid = generate_train_valid(train_path, window_size=window_size,
                                  adaptive_window=adaptive_window,
                                  valid_size=valid_ratio,
                                  sample_ratio=train_ratio,
                                  scale=scale,
                                  scale_path=scale_path,
                                  seq_len=seq_len,
                                  min_len=min_len
                                )

train_dataset = LogDataset(logkey_train, time_train, vocab, seq_len=seq_len, corpus_lines=corpus_lines, on_memory=on_memory, mask_ratio=mask_ratio)
valid_dataset = LogDataset(logkey_valid, time_valid, vocab, seq_len=seq_len, on_memory=on_memory, mask_ratio=mask_ratio)
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers,
                                  collate_fn=train_dataset.collate_fn, drop_last=True)
valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=num_workers,
                                collate_fn=train_dataset.collate_fn, drop_last=True)

# BERT model
bert = BERT(len(vocab), max_len=max_len, hidden=hidden, n_layers=layers, attn_heads=attn_heads, is_logkey=is_logkey, is_time=is_time, attn='hierarchical')
optimizer = Adam(bert.parameters(), lr=learning_rate, betas=(adam_beta1, adam_beta2), weight_decay=adam_weight_decay)
vocab_size = len(vocab)
criterion = nn.NLLLoss(ignore_index=0)
model = BERTLog(bert, vocab_size).to(device)
train_loss = []
valid_loss = []
print("Training Start")
trainer = Trainer(model, device, is_logkey=is_logkey, criterion=criterion, lr=learning_rate, optim=optimizer, train_data_loader=train_data_loader, valid_data_loader=valid_data_loader)
epochs = 10
for epoch in range(epochs):
  _, train_dist = trainer.train(epoch)
  avg_loss, valid_dist = trainer.valid(epoch)

torch.save(model.state_dict(), '../model-output/h_attn_model_dataset2.pth')

num session: 20256
before filtering short session
train size  18230
valid size  2026


100%|██████████| 20256/20256 [00:00<00:00, 289132.40it/s]

Num of train seqs 13852
Num of valid seqs 2026
Training Start



/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: U

Epoch: 1, Train Loss: 1.5343, Time: 122.08 sec


...

Epoch: 9, Valid Loss: 0.4530, Time: 29.38 sec


/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: Us

Epoch: 10, Train Loss: 0.4598, Time: 86.17 sec


/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: Us

Epoch: 10, Valid Loss: 0.4686, Time: 29.37 sec


In [15]:
model_dir = '../model-output/h_attn_model_dataset2.pth'
datapath = '../data/Loghub-HPC/processed/train'
torch_vocab = TorchVocab(datapath=datapath)
vocab = torch_vocab.get_vocab()
bert = BERT(len(vocab), max_len=max_len, hidden=hidden, n_layers=layers, attn_heads=attn_heads, is_logkey=is_logkey, is_time=is_time, attn='hierarchical')
vocab_size = len(vocab)
model = BERTLog(bert, vocab_size)
model.load_state_dict(torch.load(model_dir, map_location=torch.device('cpu')))
accuracy, P, R, F1 = evaluate(model=model, data_dir='../data/Loghub-HPC/processed/')

26732it [00:00, 269202.10it/s]
6087it [00:00, 260406.03it/s]

Testing normal data



/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: U

Testing abnormal data


/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/allison/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: Us

In [16]:
print("BERT model with Hierarchical Attention on HPC data results:")
print(f"Accuracy: {accuracy:.2f}, Precision: {P:.2f}, Recall: {R:.2f}, F1 Score: {F1:.2f}")

BERT model with Hierarchical Attention on HPC data results:
Accuracy: 81.61, Precision: 82.52, Recall: 10.47, F1 Score: 18.59
